```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Meassuring computational efficiency

This notebook will setup and run benchmarks for meassuring computational efficiency of the different schemes.
We run a simple case of Rossby adjustment with a reflective wall boundary. This should generate enough 

#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys
import time
import json
import subprocess


#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, PlotHelper, Common
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
#cl_ctx = pyopencl.create_some_context()
#print "Using ", cl_ctx.devices[0].name

In [ ]:
%env CL_LOG_ERRORS="stdout"

In [ ]:
#Create output directory for images
jsondir='computationalEfficiencyResults/'
#imgdir='images_linearCoriolis_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
#os.makedirs(imgdir)
#print "Saving images to " + imgdir

## Parameters



In [ ]:
domainsizes = range(200, 1600, 100)
globalNumSizes = len(domainsizes)

dx = 200.0
dy = 200.0

dt = 0.1

warmUpT = 40
experimentT = 1000

g = 9.81
r = 0.0

# Eddy viscocity parameter
A = 0

# Coriolis parameters: f + beta * y
f = 0.02
#beta = 2.0e-11

wind = Common.WindStressParams(type=99) # type = 99 corresponds to no wind.

waterHeight = 60

reload(Common)

def makePlot(sim, ghosts, caseName):
    nx = sim.nx
    ny = sim.ny
    dx = sim.dx
    dy = sim.dy
    eta, hu, hv = sim.download()
    fig = plt.figure(figsize=(7,7))
    #Calculate radius from center of bump for plotting
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
    x_coords = np.subtract(x_coords, x_center)
    y_coords = np.subtract(y_coords, y_center)
    radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

    plotter = PlotHelper.SinglePlot(fig, x_coords, y_coords, 
                                    eta0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], \
                                    title=caseName)

In [ ]:
### Initial condition for eta covering all of the domain

def addMultiBumps(eta, nx, ny, dx, dy, halo):
    amplitude = 0.5
    period = 25*dx
    for j in range(-halo[2], ny + halo[0]):
        y = (j+0.5)*dy
        for i in range(-halo[3], nx + halo[1]):
            x = (i+0.5)*dx
            eta[j+halo[2], i+halo[3]] += amplitude*(np.sin(x/period) + np.cos(y/period))

nx = 200
ny = 200
ghosts = [2,2,2,2]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
eta1 = np.zeros(dataShape, dtype=np.float32, order='C');
eta2 = np.zeros(dataShape, dtype=np.float32, order='C');

addBump(eta1, nx, ny, dx, dy, 0.5, 0.5, 50, ghosts)
addMultiBumps(eta2, nx, ny, dx, dy, ghosts)

fig = plt.figure(figsize=(5,5))
plt.imshow(eta1, origin="lower")
plt.colorbar()

fig = plt.figure(figsize=(5,5))
plt.imshow(eta2, origin="lower")
plt.colorbar()


In [ ]:
# Test with KP07
json_file_name = jsondir + "all_simulators_" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")  +".json"

git_hash = str.strip(subprocess.check_output(['git', 'rev-parse', 'HEAD']))

main_data = {'git_hash': git_hash, 'simulators': [], \
             'timestamp': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}


numSizes = globalNumSizes


print "-----------------------"
print "-----    KP07   -------"
print "-----------------------"

kp07_results = {'sim_name': 'KP07', 'results': []}
for i in range(numSizes):
    nx = domainsizes[i]
    ny = nx
    
    ghosts = [2,2,2,2] # north, east, south, west
    boundaryConditions = Common.BoundaryConditions()
  
    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])
    caseName = "KP07 (" + str(nx) + ", " + str(ny) + ")"
    print "--------------------"
    if nx == 900 or nx == 1300:
        print "Skipping " + caseName
        continue
    print "initializing simulator " + caseName
    
    H0 = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros(dataShape, dtype=np.float32, order='C');
    v0 = np.zeros(dataShape, dtype=np.float32, order='C');
    
    #addBump(eta0, nx, ny, dx, dy, 0.5, 0.5, 50, ghosts)
    addMultiBumps(eta0, nx, ny, dx, dy, ghosts)
    
    # Create context locally in loop
    cl_ctx = pyopencl.create_some_context()
    
    #Initialize simulator
    sim = KP07.KP07(cl_ctx, \
                    eta0, H0, u0, v0, \
                    nx, ny, \
                    dx, dy, dt, \
                    g, f, r, \
                    boundary_conditions=boundaryConditions)
    
    print "Start warm-up"
    warm_up_simt = sim.step(warmUpT)
    
    print "Start experiment"
    start_time = time.time()
    t = sim.step(experimentT)
    sim.cl_queue.finish()
    stop_time = time.time()
    time_spent = stop_time - start_time
    print "Spent " + str(time_spent) + " seconds on " + caseName
    
    simdata = {'nx': nx, 'ny': ny, 'dt': dt, 'T_end': experimentT, 'wall_clock_time': time_spent, \
               'sim.t': float(sim.t), \
               'warm_up_time_steps': warmUpT, 'warm_up_sim.t': float(warm_up_simt), \
               'timestamp_finished': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"), \
               'GPU': cl_ctx.devices[0].name}
    print simdata
    
    #makePlot(sim, ghosts, caseName)
    kp07_results['results'].append(simdata)
    sim.cleanUp()

main_data['simulators'].append(kp07_results)


print "-----------------------"
print "-----    CDKLM  -------"
print "-----------------------"


cdklm_results = {'sim_name': 'CDKLM16', 'results': []}
for i in range(numSizes):
    nx = domainsizes[i]
    ny = nx
    
    ghosts = [2,2,2,2] # north, east, south, west
    boundaryConditions = Common.BoundaryConditions()
  
    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])
    caseName = "CDKLM16 (" + str(nx) + ", " + str(ny) + ")"
    print "--------------------"
    print "initializing simulator " + caseName
    
    H0 = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros(dataShape, dtype=np.float32, order='C');
    v0 = np.zeros(dataShape, dtype=np.float32, order='C');
    
    #addBump(eta0, nx, ny, dx, dy, 0.5, 0.5, 50, ghosts)
    addMultiBumps(eta0, nx, ny, dx, dy, ghosts)
    
    # Create context locally in loop
    cl_ctx = pyopencl.create_some_context()

    #Initialize simulator
    sim = CDKLM16.CDKLM16(cl_ctx, \
                          eta0, u0, v0, H0, \
                          nx, ny, \
                          dx, dy, dt, \
                          g, f, r, \
                          boundary_conditions=boundaryConditions)
    
    print "Start warm-up"
    warm_up_simt = sim.step(warmUpT)
    
    print "Start experiment"
    start_time = time.time()
    t = sim.step(experimentT)
    sim.cl_queue.finish()
    stop_time = time.time()
    time_spent = stop_time - start_time
    print "Spent " + str(time_spent) + " seconds on " + caseName
    
    simdata = {'nx': nx, 'ny': ny, 'dt': dt, 'T_end': experimentT, 'wall_clock_time': time_spent, \
               'sim.t': float(sim.t), \
               'warm_up_time_steps': warmUpT, 'warm_up_sim.t': float(warm_up_simt), \
               'timestamp_finished': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"), \
               'GPU': cl_ctx.devices[0].name}
    print simdata
    
    #makePlot(sim, ghosts, caseName)
    
    cdklm_results['results'].append(simdata)
    sim.cleanUp()

main_data['simulators'].append(cdklm_results)


print "-----------------------"
print "-----    CTCS   -------"
print "-----------------------"


ctcs_results = {'sim_name': 'CTCS', 'results': []}
for i in range(numSizes):
    nx = domainsizes[i]
    ny = nx
    
    ghosts = [1,1,1,1] # north, east, south, west
    boundaryConditions = Common.BoundaryConditions()
  
    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])
    caseName = "CTCS (" + str(nx) + ", " + str(ny) + ")"
    print "--------------------"
    print "initializing simulator " + caseName
    
    H0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
    v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');
    
    #addBump(eta0, nx, ny, dx, dy, 0.5, 0.5, 50, ghosts)
    addMultiBumps(eta0, nx, ny, dx, dy, ghosts)
    
    ### RESET opencl context
    #https://github.com/inducer/pyopencl/blob/master/src/c_wrapper/context.cpp
    # release context called in destructor
    # Use local variable!
    
    # Create context locally in loop
    cl_ctx = pyopencl.create_some_context()

    #Initialize simulator
    sim = CTCS.CTCS(cl_ctx, \
                    H0, eta0, u0, v0, \
                    nx, ny, \
                    dx, dy, dt, \
                    g, f, r, A, \
                    boundary_conditions=boundaryConditions)

    print "Start warm-up"
    warm_up_simt = sim.step(warmUpT)
    
    print "Start experiment"
    # Add something sim.{openCL sync med cl_ctx --- flush() eller finish()}
    start_time = time.time()
    t = sim.step(experimentT)
    # Add something sim.{openCL sync med cl_ctx --- flush() eller finish()}
    sim.cl_queue.finish()
    stop_time = time.time()
    time_spent = stop_time - start_time
    print "Spent " + str(time_spent) + " seconds on " + caseName
    
    simdata = {'nx': nx, 'ny': ny, 'dt': dt, 'T_end': experimentT, 'wall_clock_time': time_spent, \
               'sim.t': float(sim.t), \
               'warm_up_time_steps': warmUpT, 'warm_up_sim.t': float(warm_up_simt), \
               'timestamp_finished': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"), \
               'GPU': cl_ctx.devices[0].name}
    print simdata
    
    #makePlot(sim, ghosts, caseName)
    
    ctcs_results['results'].append(simdata)
    sim.cleanUp()
    
main_data['simulators'].append(ctcs_results)


print "-----------------------"
print "-----    FBL    -------"
print "-----------------------"

#numSizes = 6

fbl_results = {'sim_name': 'FBL', 'results': []}
for i in range(numSizes):
    nx = domainsizes[i]
    ny = nx
    
    ghosts = [0,0,0,0] # north, east, south, west
    validDomain = [None, None, 0, 0]    
    boundaryConditions = Common.BoundaryConditions()
  
    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])
    caseName = "FBL (" + str(nx) + ", " + str(ny) + ")"
    print "--------------------"
    print "initializing simulator " + caseName
    
    H0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
    v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');
    
    #addBump(eta0, nx, ny, dx, dy, 0.5, 0.5, 50, ghosts)
    addMultiBumps(eta0, nx, ny, dx, dy, ghosts)

    
    # Create context locally in loop
    cl_ctx = pyopencl.create_some_context()

    #Initialize simulator
    sim = FBL.FBL(cl_ctx, \
                  H0, eta0, u0, v0, \
                  nx, ny, \
                  dx, dy, dt, \
                  g, f, r, \
                  boundary_conditions=boundaryConditions)

    print "Start warm-up"
    warm_up_simt = sim.step(warmUpT)
    
    print "Start experiment"
    start_time = time.time()
    t = sim.step(experimentT)
    sim.cl_queue.finish()
    stop_time = time.time()
    time_spent = stop_time - start_time
    print "Spent " + str(time_spent) + " seconds on " + caseName
    
    simdata = {'nx': nx, 'ny': ny, 'dt': dt, 'T_end': experimentT, 'wall_clock_time': time_spent, \
               'sim.t': float(sim.t), \
               'warm_up_time_steps': warmUpT, 'warm_up_sim.t': float(warm_up_simt), \
               'timestamp_finished': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"), \
               'GPU': cl_ctx.devices[0].name}
    print simdata
    
    #makePlot(sim, validDomain, caseName)
    
    fbl_results['results'].append(simdata)
    sim.cleanUp()
    
main_data['simulators'].append(fbl_results)




print main_data
with open(json_file_name, 'w') as fout:
    json.dump(main_data, fout)

In [ ]:
#main_data['simulators'].append(fbl_results)
#with open(json_file_name, 'w') as fout:
#    json.dump(main_data, fout)
a = []
a.append(1)
a.append(2)
print a
print datetime.datetime.now()

In [ ]:
print json_file_name